In [1]:
import torch
from torchvision import datasets, transforms
# from torchvision.transforms import v2
from torch.utils.data import DataLoader, Dataset, Subset, random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as  np
from torchvision import utils
import matplotlib.pyplot as plt
import os

In [2]:
# !unzip "Train_dataset_small.zip" -d "dataset"

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
transform = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

In [5]:
dataset = datasets.ImageFolder('/user/ssheri/train_dataset',transform=transform)

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [7]:
from torch.utils.data import random_split, DataLoader

In [8]:
train_size = int(0.7 * len(dataset))
rem = len(dataset) - train_size
test=int(rem/2)


In [9]:
train_size= train_size +1

In [10]:

train_data, test_data,val_data = random_split(dataset, [train_size,test,test])

In [11]:
train_loader = DataLoader(train_data, batch_size=35, shuffle=True)
test_loader = DataLoader(test_data, batch_size=35, shuffle=False)
val_loader = DataLoader(val_data,batch_size = 35, shuffle = False)

In [12]:
len(dataset.classes)

8

In [13]:
dataset.classes

['airport_terminal',
 'auditorium',
 'bedroom',
 'bookstore',
 'bus_station-indoor',
 'clothing_store',
 'computer_room',
 'food_court']

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

class AlaxNet_custom(nn.Module):
    def __init__(self, in_channels, out_classes):
        super(AlaxNet_custom, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=64, kernel_size=11, stride=4, padding=2)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=192, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm2d(192)
        self.conv3 = nn.Conv2d(in_channels=192, out_channels=384, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(384)
        self.conv4 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        
        self.relu = nn.ReLU(inplace=True)
        self.maxPool = nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
        self.dropout = nn.Dropout(p=0.5)
        self.dropout_conv = nn.Dropout2d(p=0.1)  # New dropout for convolutional layers
        self.fc1 = nn.Linear(in_features=9216, out_features=4096)
        self.fc2 = nn.Linear(in_features=4096, out_features=4096)
        self.fc3 = nn.Linear(in_features=4096, out_features=out_classes)

    def forward(self, x):
        img = self.bn1(self.conv1(x))
        img = self.relu(img)
        img = self.maxPool(img)
        
        img = self.bn2(self.conv2(img))
        img = self.relu(img)
        img = self.maxPool(img)


        img = self.bn3(self.conv3(img))
        img = self.relu(img)
        

        img = self.bn4(self.conv4(img))
        img = self.relu(img)
        

        img = self.bn5(self.conv5(img))
        img = self.relu(img)
        img = self.maxPool(img)


        img = torch.flatten(img, 1)


        img = self.fc1(img)
        img = self.relu(img)

        img = self.fc2(img)
        img = self.relu(img)

       
        img = self.fc3(img)
        return img


# Create an instance of the model
model = AlaxNet_custom(3, 8).to(device)
# model.to(device)

# Print the model architecture
print(model)

# Reference: https://blog.paperspace.com/alexnet-pytorch/
# Reference: https://github.com/dansuh17/alexnet-pytorch/blob/d0c1b1c52296ffcbecfbf5b17e1d1685b4ca6744/model.py#L40

AlaxNet_custom(
  (conv1): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (bn2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxPool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.

In [15]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, n_epochs):
    train_losses, train_accs = [], []
    val_losses, val_accs = [], []

    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        train_loss = train_loss / len(train_loader)
        train_acc = (train_correct / train_total) * 100
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        if scheduler:
            scheduler.step()

        # Validation
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        val_loss = val_loss / len(val_loader)
        val_acc = (val_correct / val_total) * 100
        val_losses.append(val_loss)
        val_accs.append(val_acc)

        print(f'Epoch {epoch+1}/{n_epochs}:')
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

    return train_losses, train_accs, val_losses, val_accs

In [16]:
# criterion = nn.CrossEntropyLoss().to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

In [17]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [18]:
# model = ResNet18(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

In [19]:

num_epochs = 60
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []
patience = 4
best_loss = float('inf')

# Training loop
for epoch in range(num_epochs):
    model.train()
    training_loss = 0.0
    total = 0
    correct = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        training_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    training_loss /= len(train_loader)
    train_losses.append(training_loss)
    train_accuracy = 100 * correct / total
    train_accuracies.append(train_accuracy)

    # Validation
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for data in val_loader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
    val_loss /= len(val_loader)
    val_accuracy = 100 * correct_val / total_val
    
    scheduler.step(val_loss)
    
    if val_loss<best_loss:
        best_loss = val_loss
        counter = 0
    else:
        counter = 1
    if counter == patience:
        print('early stopping occured')
        torch.save(model.state_dict(), 'resetearly.pth')
        break
    
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    print(f"Epoch [{epoch+1}/{num_epochs}] Train Loss: {training_loss:.4f},Train_accuracy : {train_accuracy:.4f}%, Val Loss: {val_loss:.4f}, Val_accuracy: {val_accuracy:.4f}%")
#     if val_accuracy > 77:
#         savepath = f"/kaggle/working/rnet_model_accuracy_{val_accuracy}.pth"
#         torch.save(model.state_dict(), savepath)
    

print('Finished Training')

Epoch [1/60] Train Loss: 2.0240,Train_accuracy : 29.5354%, Val Loss: 1.7600, Val_accuracy: 35.5764%
Epoch [2/60] Train Loss: 1.6547,Train_accuracy : 38.9471%, Val Loss: 1.7506, Val_accuracy: 38.0108%
Epoch [3/60] Train Loss: 1.5106,Train_accuracy : 45.2699%, Val Loss: 1.8934, Val_accuracy: 36.8979%
Epoch [4/60] Train Loss: 1.3792,Train_accuracy : 50.7918%, Val Loss: 1.3004, Val_accuracy: 54.3384%
Epoch [5/60] Train Loss: 1.2859,Train_accuracy : 54.4618%, Val Loss: 1.2891, Val_accuracy: 54.3036%
Epoch [6/60] Train Loss: 1.2279,Train_accuracy : 57.1333%, Val Loss: 1.2536, Val_accuracy: 56.4597%
Epoch [7/60] Train Loss: 1.1802,Train_accuracy : 59.0149%, Val Loss: 1.2314, Val_accuracy: 56.5119%
Epoch [8/60] Train Loss: 1.1419,Train_accuracy : 60.5164%, Val Loss: 1.1737, Val_accuracy: 59.9548%
Epoch [9/60] Train Loss: 1.1111,Train_accuracy : 61.6044%, Val Loss: 1.1667, Val_accuracy: 60.6503%
Epoch [10/60] Train Loss: 1.0851,Train_accuracy : 62.1409%, Val Loss: 1.1179, Val_accuracy: 61.2589%

In [ ]:
# train_losses, train_accs, val_losses, val_accs = train_model(model, train_loader, val_loader, criterion, optimizer,scheduler, 50)

Epoch 1/50:
Train Loss: 0.8251, Train Acc: 71.7836
Val Loss: 0.7909, Val Acc: 72.8221
Epoch 2/50:
Train Loss: 0.8176, Train Acc: 71.8134
Val Loss: 0.8057, Val Acc: 72.9091
Epoch 3/50:
Train Loss: 0.8246, Train Acc: 71.6532
Val Loss: 0.7843, Val Acc: 74.0393
Epoch 4/50:
Train Loss: 0.8238, Train Acc: 71.7165
Val Loss: 0.8067, Val Acc: 72.5961
Epoch 5/50:
Train Loss: 0.8179, Train Acc: 71.9066
Val Loss: 0.7874, Val Acc: 73.1351
Epoch 6/50:
Train Loss: 0.8272, Train Acc: 71.3998
Val Loss: 0.7942, Val Acc: 73.3438
Epoch 7/50:
Train Loss: 0.8274, Train Acc: 71.5563
Val Loss: 0.8035, Val Acc: 72.8395
Epoch 8/50:
Train Loss: 0.8302, Train Acc: 71.4296
Val Loss: 0.7972, Val Acc: 73.0308
Epoch 9/50:
Train Loss: 0.8174, Train Acc: 71.8060
Val Loss: 0.7969, Val Acc: 72.8743
Epoch 10/50:
Train Loss: 0.8187, Train Acc: 71.6420
Val Loss: 0.7962, Val Acc: 72.7178
Epoch 11/50:
Train Loss: 0.8241, Train Acc: 71.5153
Val Loss: 0.7816, Val Acc: 73.8132
Epoch 12/50:
Train Loss: 0.8304, Train Acc: 71.5638


In [ ]:
# torch.save(model.state_dict(), 'alaxnet_final_project_60.pth')

In [ ]:
model.load_state_dict(torch.load('alaxnet_final_project_60.pth'))
model = model.to(device)

In [ ]:
def eval_model(model, data_loader, type='validation'):
  correct=0
  total=0
  running_loss = 0.0
  with torch.no_grad():
    model.eval()
    for inputs, labels in data_loader:
      inputs, labels = inputs.to(device), labels.to(device)
      pred = model(inputs)
      total += labels.size(0)
      loss = criterion(pred, labels.squeeze().long())
      running_loss += loss.item()
      _, predicted = torch.max(pred.data, 1)
      correct += predicted.eq(labels.squeeze().long()).sum().item()
    # print(f'Accuracy of {type},  {100 * correct // total} %')
  test_val_loss = running_loss / len(data_loader)
  test_val_accuracy = 100 * correct / total
  return test_val_loss, test_val_accuracy

In [ ]:
test_loss, test_accuracy = eval_model(model, test_loader, type='testing')

In [ ]:
print(test_loss, test_accuracy)

In [ ]:
import cv2

image = cv2.imread('00000012.jpg')
image_size = 224  # Assuming this matches your training image size
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

resized = cv2.resize(image, (image_size, image_size))
normalized = (resized / 255.0 - mean)


In [ ]:
transformed_image = torch.from_numpy(normalized.astype(np.float32)).permute(2, 0, 1)
transformed_image = transformed_image.unsqueeze(0) 
transformed_image = transformed_image.to(device)

In [43]:
model.eval()
with torch.no_grad():  # Disable gradient calculation for efficiency
    outputs = model(transformed_image)
    _, predicted = torch.max(outputs.data, 1)
    class_names = ['airport_terminal',
                     'auditorium',
                     'bedroom',
                     'bookstore',
                     'bus_station-indoor',
                     'clothing_store',
                     'computer_room',
                     'food_court',
                     'jewelry_shop',
                     'railroad_track']
    predicted_class = class_names[predicted.item()]
    print(f'Predicted class: {predicted_class}')


Predicted class: airport_terminal


In [39]:
predicted.item()

0